In [1]:
import os
import openai
import json

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
def get_day_of_week(days_ahead=0):
    import datetime
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    future_day = (datetime.datetime.now() + datetime.timedelta(days=days_ahead)).weekday()
    return days[future_day] 

In [26]:
def run_conversation(days_input):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "{}".format(days_input)}]
    functions = [
        {
            "name": "get_day_of_week",
            "description": "Get the day of week a number of days after today",
            "parameters": {
                "type": "object",
                "properties": {
                    "days_ahead": {
                        "type": "integer",
                        "description": "The number of days after the system's current time",
                    }
                },
                "required": ["days_ahead"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
        temperature=0.0,
    )
    print(response)
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_day_of_week": get_day_of_week,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            days_ahead=function_args.get("days_ahead")
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            temperature=0.0,
        )  # get a new response from GPT where it can see the function response
    
    else:
        second_response = response
    
    return second_response

In [27]:
response1 = run_conversation('What day is it today?')

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"days_ahead\": 0\n}",
          "name": "get_day_of_week"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1687386072,
  "id": "chatcmpl-7U0UKlYPx7QfP9leD924xTRqgMC0H",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 73,
    "total_tokens": 92
  }
}


In [28]:
print(response1)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Today is Wednesday.",
        "role": "assistant"
      }
    }
  ],
  "created": 1687386073,
  "id": "chatcmpl-7U0ULlPrBrPBAoN4gwOTlur68oMHE",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 4,
    "prompt_tokens": 43,
    "total_tokens": 47
  }
}


In [29]:
print(response1['choices'][0]['message']['content'])

Today is Wednesday.


In [19]:
print(run_conversation('What day is 11 days from now?')['choices'][0]['message']['content'])

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"days_ahead\": 11\n}",
          "name": "get_day_of_week"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1687385755,
  "id": "chatcmpl-7U0PDk4Je7UMHBwEWrIesygE4s7Ue",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 76,
    "total_tokens": 95
  }
}
11 days from now will be a Sunday.


In [30]:
print(run_conversation('Where is San Francisco located?')['choices'][0]['message']['content'])

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "San Francisco is located in the state of California, on the west coast of the United States.",
        "role": "assistant"
      }
    }
  ],
  "created": 1687386107,
  "id": "chatcmpl-7U0UttkrAd9PmJZdZEqKotRz7LQSt",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 20,
    "prompt_tokens": 73,
    "total_tokens": 93
  }
}
San Francisco is located in the state of California, on the west coast of the United States.


In [31]:
print(run_conversation('What day is October 12, 2023')['choices'][0]['message']['content'])

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"days_ahead\": 731\n}",
          "name": "get_day_of_week"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1687386116,
  "id": "chatcmpl-7U0V2nVICrcCQoXZFutRvEMq1ddlq",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 77,
    "total_tokens": 96
  }
}
October 12, 2023 falls on a Saturday.
